In [ ]:
#install all the necessary libraries
!pip install -q Pypdf2 langchain langchain-google-genai faiss-gpu nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
#mount google drive to load documents
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import all the important packages
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import string
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
#enter your gemini api key
genai.configure(api_key='')
os.environ["GOOGLE_API_KEY"]=""

In [ ]:
#returns text from all the documents
def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf,strict=False)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text

In [ ]:
#preprocessing the texts
def text_preprocessing(text:str):
    #convert lower case and remove all the links and citations
    text=text.lower()
    text=text.replace('\n',' ')
    text=re.sub(r'www\S+',' ',text)
    text=re.sub(r'http\S+',' ',text)
    text=re.sub(r'\S+@\S+',' ',text)
    text=re.sub(r'[^a-zA-Z0-9\s]',' ',text)
    text=re.sub(r'\[0-9,\]','',text)

    #removing the sentences which do not contain any stopwords
    temp=nltk.sent_tokenize(text)
    k=[]
    for i in temp:
        for word in i:
            if word in stopwords.words('english'):
                k.append(i)
                break

    text=' '.join(k)

    text=nltk.word_tokenize(text)

    #removing all the digits and alphanumeric words
    y=[]
    for i in text:
        #if i.isdigit():
            #y.append(i)
        #elif i.isalnum():
            #y.append(i)
        if i.isalpha():
            if i in words.words():
                y.append(i)

    text.clear()
    #remove stopwords and punctuation
    for i in y:
        if i not in stopwords.words('english') and i not in string.punctuation:
            text.append(i)
    y.clear()

    #applying stemming
    ps=PorterStemmer()
    for i in text:
        y.append(ps.stem(i))

    text=" ".join(y)
    text=re.sub('\s[a-z]\s','',text)
    return text

In [ ]:
#dividing text into chunks
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
#storing data in FAISS after converting into embeddings
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("temp")

In [ ]:
#loading the model with prompt
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.5,safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    })

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [ ]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

    #retiriving similar data from the FAISS database
    new_db = FAISS.load_local("temp", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question,k=5)

    chain = get_conversational_chain()

    #input in gemini
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)

In [ ]:
pdf_docs=['/content/drive/MyDrive/Cognite/Abbas_2018.pdf','/content/drive/MyDrive/Cognite/1_Ramırez-Duque_.pdf','/content/drive/MyDrive/Cognite/15_Nazneen.pdf','/content/drive/MyDrive/Cognite/Abbas_2020.pdf'
,'/content/drive/MyDrive/Cognite/LEE.pdf','/content/drive/MyDrive/Cognite/carpenter2020 (1).pdf','/content/drive/MyDrive/Cognite/zhao2020.pdf','/content/drive/MyDrive/Cognite/Tariq_2019.pdf','/content/drive/MyDrive/Cognite/Asd_Cry_patterns.pdf'
,'/content/drive/MyDrive/Cognite/Patten_Audio.pdf','/content/drive/MyDrive/Cognite/Dawson.pdf','/content/drive/MyDrive/Cognite/22_Ouss_ASD.pdf','/content/drive/MyDrive/Cognite/Tariq2018.pdf','/content/drive/MyDrive/Cognite/Qiu.pdf'
,'/content/drive/MyDrive/Cognite/Young_Behavior.pdf']

raw_text = get_pdf_text(pdf_docs)


In [ ]:
processed_text=text_preprocessing(raw_text)
text_chunks = get_text_chunks(processed_text)
get_vector_store(text_chunks)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
new_db = FAISS.load_local("temp", embeddings, allow_dangerous_deserialization=True)

Query 1

In [ ]:
new_db.similarity_search('What are the variety of Multimodal and Multi-modular AI Approaches to Streamline Autism Diagnosis in Young Children',k=5)

[Document(page_content='research machin learn approach earli detect autism combin questionnair home video screen abba gloveralto ca pediatr univers ca data scienc univers ca correspond alto ca receiv march editori decis march accept abstract background screen earli detect autism expens cumbersom time intens sometim fall short predict valu work sought appli machin learn clinic data across risk autism spectrum disord creat alow cost quick easi appli autism screen tool two train identifi autism one base short structur parent key short semi structur home binat algorithm use combin singl assess higher accuraci come scarciti sparsiti imbal train data appli novel featur select featur allow inconclus determin appropri boost screen accuraci conclus perform center clinic studiascertain perform combin demonstr cant accuraci improv standard screen ing sensit citi conclus suggest mobil machin learn process reliabl method detect autism outsid clinic varieti confound clinic analysi final statist limi

In [ ]:
query='What are the variety of Multimodal and Multi-modular AI Approaches to Streamline Autism Diagnosis in Young Children'
user_input(text_preprocessing(query))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'output_text': "- **Modular machine learning-based assessment of autism:** This approach involves using machine learning algorithms to analyze data from multiple sources, such as parent reports, clinician observations, and home videos, to assess autism symptoms. This can help to streamline the diagnostic process and make it more efficient.\n\n- **Three complementary, unified outcomes:** This approach combines the results of the three different assessment methods (parent report, clinician observation, and home video) to create a more comprehensive and reliable diagnosis.\n\n- **Reliable in minutes:** This approach can provide a reliable diagnosis in just a few minutes, which can significantly reduce the time it takes to get a diagnosis.\n\n- **Parent report questionnaire via mobile:** This approach allows parents to complete a questionnaire about their child's symptoms on their mobile phone, which can make the diagnostic process more convenient and accessible.\n\n- **Semi-structured ho

#Query 2

In [ ]:
new_db.similarity_search('What is Autism Spectrum Disorder, how it is caysed?',k=5)

[Document(page_content='exampl articl diagnos autism facial autism facial express diagnosi autism develop autism disord collect autism spectrum disord al sinc child psychiatrist first autism incid autism risen rapidli everi peopl diseas everi diseas accord nation center health statist probabl autism among age unit percent al statist survey autist china howev accord data report develop china autism educ rehabilit industri number peopl autism china exceed million million autist time grow rate nearli annual deer autism research institut autism seriou financi burden societi famili autist one hand spend lot time work reduc work incom reduc hand cost famili rehabilit treatment autist huge famili financi burden accord survey occup econom burden preschool autist percent autist serious affect annual incom significantli lower ordinari averag loss incom yuan per year meanwhil averag annual cost autist educ train significantli higher ordinari yang wang societi govern also need invest lot money reh

In [ ]:
query='What is Autism Spectrum Disorder, how it is caysed?'
user_input(text_preprocessing(query))

{'output_text': "Autism spectrum disorder (ASD) is a complex developmental disorder that affects a person's ability to communicate and interact with others. It is characterized by difficulties with social interaction, repetitive behaviors, and restricted interests. ASD can range in severity from mild to severe, and it can affect people of all ages.\n\nThe exact cause of ASD is unknown, but it is thought to be caused by a combination of genetic and environmental factors. ASD is often diagnosed in early childhood, but it can also be diagnosed in adulthood. There is no cure for ASD, but there are treatments that can help to improve symptoms and quality of life."}


Query 3

In [ ]:
new_db.similarity_search('What is the cure of Autism Spectrum Disorder',k=5)

[Document(page_content='autism development morganself regul affect express play autism syndrom short term longitudin studi journal developmentggoldtrace tempor autism spectrum diagnosi sever measur diagnost observ schedul systemat meta analysi onecarpentlborgg comput vision analysi atyp attent autism autismcarpentljg use digit autism follow improv qualiti screen autism pediatrhalln swain techali whitegroup facial emot express autism evid machin cation behavior therapi changc linj librari support vector intellig technologjvjpreval autism spectrum disord among chil age autism development network unitdyadic triadic infanc later social respons autist disord journal autism carpent al digit behavioryoungp earli autist disord video analysi journal autism developmentearli develop joint attent autist disord parent interview journal developmentreduc facial muscl move autism evid dysfunct pathway brain cognitr quarter centuri progress earli detect treatment autism developkj smithcarpentg atyp con

In [ ]:
query='What is the cure of Autism Spectrum Disorder'
user_input(text_preprocessing(query))

{'output_text': 'The provided context does not mention anything about curing autism spectrum disorder, so I cannot answer this question from the provided context.'}


Query4

In [ ]:
new_db.similarity_search('What are Stereotypical and maladaptive behaviors in Autism Spectrum, how are these detected and managed',k=5)

[Document(page_content='progress earli detect treatment autism developkj smithcarpentg atyp control via comput vision analysi autism spectrum disordg hill spencerl affect young autist journal abnorm child psychologg potenti digit behavior measur transform detect autism spectrum disord jama pediatrkjj earli social attent autism social joint attent attent development psycholog de lawf dingj intern confer automat face gestur recognittwo way street automat emot moral journal experiment child eggerlj carpentlkautomat emot attent analysi young home autism feasibl studi digit medicinreveal new york timeface basic appli stud spontan express facial action new york univers presshlnposit affect infant autism journal abnorm child psychologc random sampl paradigm model imag analysi cartographi machinerideceit facial express role poker face child depend person detector psychologljoint initi without posit affect risk group journal developmentlordautism diagnost observ schedul diagnost valid journal a

In [ ]:
query='What are Stereotypical and maladaptive behaviors in Autism Spectrum, how are these detected and managed'
user_input(text_preprocessing(query))

{'output_text': 'The provided context does not contain any information about stereotypes of autism spectrum disorder, so I cannot answer this question from the provided context.'}


Query5

In [ ]:
new_db.similarity_search('How relevant is eye contact and how it can be used to detect Autism',k=5)

[Document(page_content='mother infant interact affectal development hand typic risk development observ studi year life frontal take account infant engag emot earli may help determin risk autism intellectu disabl west syndrom child psychiatribrunetr vi de de la premier lordrutter autism diagnost interview version diagnost interview pervas devautism devrrk toward object cation childhood autism childhood autism rate scaleautism devbcolor base particl joint detect multipl acoust speech signal pa huede mont pour non lin air un bruit laneunehe de blake condens condit densiti propag visualvicfknbeneellverecognit technolog method spontan high risk front al dynam non verbal father infant interact affectal linguist employ complex commun engag social repair interactsoc open saintal recogn autist deviant behavior long diagnosi take account interact comput one saintal interact cross road emot cognit systemat review one al home video detect may help studi interact becom autiste ibap induct one level

In [ ]:
query='How relevant is eye contact and how it can be used to detect Autism'
user_input(text_preprocessing(query))

{'output_text': 'The duration of active social engagement was positively correlated with the social commun factor score, language factor score, and the total csb dp score. Frequency of eye contact was positively correlated with the social commun factor score, the language factor score, and the total csb dp score.'}


Query6

In [ ]:
new_db.similarity_search('How can cross country trials help in development of Machine learning based Multimodal solutions',k=5)

[Document(page_content='train framework comparison model linear kernel train data split deep learn model well vanilla singl eightpeer review figur structur joint optim model auto ae bidirect long short term memori perform evalu perform method five fold cross valid averag averag proport distribut five estim utter data five valid model tabl use train model selecttest ae joint model classif stage one utter frame wise method outputclass indic averag class indic utter consid child utter score convent well unweight averag recalweight averag recal war chosen emot challeng consid class experi model low precis extrem toward class er ae model consider qualiti ae model margin improv correctli select degrad qualiti toward tabl classif support vector machin selectae ae accuraci precis recal score unweight averag recal figur structur joint optim model auto ae bidirect long short term memori perform evalu perform method fold cross valid averag averag proport distribut gener estim utter data valid mod

In [ ]:
query='How can cross country trials help in development of Machine learning based Multimodal solutions'
user_input(text_preprocessing(query))

{'output_text': 'The provided context does not mention anything about cross country helping develop machine learning based multimodal, so I cannot answer this question from the provided context.'}


Query7

In [ ]:
new_db.similarity_search('How early infants cry can help in the early detection of Autism',k=5)

[Document(page_content='research machin learn approach earli detect autism combin questionnair home video screen abba gloveralto ca pediatr univers ca data scienc univers ca correspond alto ca receiv march editori decis march accept abstract background screen earli detect autism expens cumbersom time intens sometim fall short predict valu work sought appli machin learn clinic data across risk autism spectrum disord creat alow cost quick easi appli autism screen tool two train identifi autism one base short structur parent key short semi structur home binat algorithm use combin singl assess higher accuraci come scarciti sparsiti imbal train data appli novel featur select featur allow inconclus determin appropri boost screen accuraci conclus perform center clinic studiascertain perform combin demonstr cant accuraci improv standard screen ing sensit citi conclus suggest mobil machin learn process reliabl method detect autism outsid clinic varieti confound clinic analysi final statist limi

In [ ]:
query='How early infants cry can help in the early detection of Autism'
user_input(text_preprocessing(query))

{'output_text': 'This study found that computer vision analysis of facial movement expression via tablet-based assessment can detect differences in affect expression, one of the early core features of ASD.'}


Query8

In [ ]:
new_db.similarity_search('What are various methods to detect Atypical Pattern of Facial expression in Children',k=5)

[Document(page_content='al automat track facial child face de la al head rel camera optim rotat canon face model visibl tag assign face drastic yaw center acknowledg current method use whether child toward stimulu track eye visibl frame probabl three standard facial posit neutralactiv facial figur exampl movi stimuli development appropri mechan bunni anim split screen show woman sing nurseri one side dynam nois side carpent al digit behavior unit assign al model express extens pose invari cross modal dictionari learn approach origin al train represent setup map facial abl infer discrimin facial inform facial recognit even facial avail deploy train data bing univers facial express yin sun wang use along see al synthesi extract imag subset facial use facial learn robust dictionari lastli discrimin facial support vector machin chang lin train differ facial recent progress automat facial express analysi al thrasher chow addit algorithm expert human rater facial affect subsampl video record

In [ ]:
query='What are various methods to detect Atypical Pattern of Facial expression in Children'
user_input(text_preprocessing(query))

{'output_text': 'The various methods used to detect atypical patterns of facial expression in children include:\n\n- Computer vision analysis (CVA)\n- Automatic detection and tracking of facial landmarks\n- Estimation of head position\n- Facial expression positivity\n- Neutral expression'}


Query9

In [ ]:
new_db.similarity_search('What kind of facial expressions can be used to detect Autism Disorder in children',k=5)

[Document(page_content='standard facial express studi recognit differ emot autist found autist better recognit basic facial happi rel complex facial surpris recognit difficult recogn present main diagnost criteria autism autism child behavior scale autism rate scale behavior scale wang consult larg number investig actual situat hospit hospit mainli fill fill fill diagnos autism detail review test three scale scale contain test judg autism facial scale seventh item inexplic laughter tenth item look face eye contact relat express scale seventh item non commun smile seventeenth item respond peopl facial twenti fourth item activ avoid eye contact fifteen scale third emot respons pleasur unhappi interest express facial express postur scale basic includ autism detect facial show diagnosi autism accur facial progress artifici intellig technolog facial express recognit technolog object effect reflect mental health use earli diagnosi autism al also matern child health hospit hospit medic center

In [ ]:
query='What kind of facial expressions can be used to detect Autism Disorder in children'
user_input(text_preprocessing(query))

{'output_text': 'The paper uses the latest deep learning technology to improve the accuracy of facial expression recognition, which is better than the previous traditional techniques. The low recognition rate of facial expression in recent years has been improved by the development of artificial intelligence technology. The great improvement in the accuracy of facial expression recognition by convolutional neural networks has greatly improved the accuracy of facial expression recognition.'}


Query10

In [ ]:
new_db.similarity_search('What are methods to detect Autism from home videos',k=5)

[Document(page_content='research machin learn approach earli detect autism combin questionnair home video screen abba gloveralto ca pediatr univers ca data scienc univers ca correspond alto ca receiv march editori decis march accept abstract background screen earli detect autism expens cumbersom time intens sometim fall short predict valu work sought appli machin learn clinic data across risk autism spectrum disord creat alow cost quick easi appli autism screen tool two train identifi autism one base short structur parent key short semi structur home binat algorithm use combin singl assess higher accuraci come scarciti sparsiti imbal train data appli novel featur select featur allow inconclus determin appropri boost screen accuraci conclus perform center clinic studiascertain perform combin demonstr cant accuraci improv standard screen ing sensit citi conclus suggest mobil machin learn process reliabl method detect autism outsid clinic varieti confound clinic analysi final statist limi

In [ ]:
query='What are methods to detect Autism from home videos'
user_input(text_preprocessing(query))

{'output_text': 'The study moves beyond verbal descriptions and employs video examples to reduce subject interpretation. Use of videos has shown a dramatic increase in clarity in fields such as music instruction and motor vehicle repair. Recent videos have been incorporated into ASD screening to reduce discrepancies in interpretation of items and provide information common to naturalist standard comparisons.'}


Query11

In [ ]:
new_db.similarity_search('What is Still-Face Paradigm in Early Screening for High-Risk Autism Spectrum Disorder',k=5)

[Document(page_content='research machin learn approach earli detect autism combin questionnair home video screen abba gloveralto ca pediatr univers ca data scienc univers ca correspond alto ca receiv march editori decis march accept abstract background screen earli detect autism expens cumbersom time intens sometim fall short predict valu work sought appli machin learn clinic data across risk autism spectrum disord creat alow cost quick easi appli autism screen tool two train identifi autism one base short structur parent key short semi structur home binat algorithm use combin singl assess higher accuraci come scarciti sparsiti imbal train data appli novel featur select featur allow inconclus determin appropri boost screen accuraci conclus perform center clinic studiascertain perform combin demonstr cant accuraci improv standard screen ing sensit citi conclus suggest mobil machin learn process reliabl method detect autism outsid clinic varieti confound clinic analysi final statist limi

In [ ]:
query='What is Still-Face Paradigm in Early Screening for High-Risk Autism Spectrum Disorder'
user_input(text_preprocessing(query))

{'output_text': "The still face paradigm (SFP) is a behavioral observation method used to assess infant emotion regulation and social expectations in social interactions. It involves three episodes: a baseline episode, a face-to-face interaction episode, and a reunion episode. During the face-to-face interaction episode, the mother is required to present a neutral expression without any response to the child's actions. This causes the child to experience a disruption in social signals, which can lead to the appearance of negative emotions. The SFP has been used in research to study the social and emotional development of infants and children, including those at high risk for autism spectrum disorder (ASD)."}


Query12

In [ ]:
new_db.similarity_search('What is West Syndrome?',k=5)

[Document(page_content='omksolokkqkkgw wo omstz yoq owyo omqq st ow zovzlomqstzzw omzo owzmovmoomkq owstsow mokk molqkzq yokwzoomyouowl soy yonz om wozow lo sokz zoqk lozwzqzzzqnzkqq sotz stfzzkzwowozwk wozwzzwowwkwz wow zo mo wowozwkwwkkk wowk wowwowozw omyouowqqkomowknjow wokrgzj woz omowkwojom sok owowfj omkqom wonlqnkwonkk owzkq ortow yoqowzjfgofwwwozzz om jokgn frok om wo noykzlozomq owzlstzwz omnfkrwoqomz wokom joznz wo owkomyouq zorw zoufq wo mowzzqqqwwqom zonn owzjkz owvyof jojlzk omwrqk omznulq omrlllzzq jowr jowllzgkkfzkg zonkwk jog zonkwqwgzozzq owjowkqlv kylww jo omr silolkqlwbzzolgon zokn zo om omyouzn zost lokwom fun mon om zo zockkzokkwn omyou research june pediatr june volum articl lori canada correspond receiv accept citat risk front still face paradigm earli screen high risk autism spectrum disord china medic engin chinachin china background although autism spectrum disord current di age age diagnosi still en later order earli recent method still face paradigm adopt m

In [ ]:
query='What is West Syndrome?'
user_input(text_preprocessing(query))

{'output_text': 'West syndrome (WS) is a rare epileptic encephalopathy with early onset before age 1 year and a high risk of neurodevelopmental disorders (NDD) outcomes, including one third of WS children showing later autism spectrum disorder (ASD) and/or intellectual disability (ID).'}


Query13

In [ ]:
new_db.similarity_search('What is the utility of Behavior and interaction imaging at 9 months of age predict autism/intellectual disability in high-risk infants with West syndrome',k=5)

[Document(page_content='mother infant interact affectal development hand typic risk development observ studi year life frontal take account infant engag emot earli may help determin risk autism intellectu disabl west syndrom child psychiatribrunetr vi de de la premier lordrutter autism diagnost interview version diagnost interview pervas devautism devrrk toward object cation childhood autism childhood autism rate scaleautism devbcolor base particl joint detect multipl acoust speech signal pa huede mont pour non lin air un bruit laneunehe de blake condens condit densiti propag visualvicfknbeneellverecognit technolog method spontan high risk front al dynam non verbal father infant interact affectal linguist employ complex commun engag social repair interactsoc open saintal recogn autist deviant behavior long diagnosi take account interact comput one saintal interact cross road emot cognit systemat review one al home video detect may help studi interact becom autiste ibap induct one level

In [ ]:
query='What is the utility of Behavior and interaction imaging at 9 months of age predict autism/intellectual disability in high-risk infants with West syndrome'
user_input(text_preprocessing(query))

{'output_text': 'Yes, the study found that multi-modal social signal interaction and synchrony in infant-mother interaction at 9 months of age could predict the outcome of autism spectrum disorder (ASD) and intellectual disability (ID) in high-risk infants with West syndrome (WS).'}
